In [1]:
# find output/ -type f -name "*.csv" -newermt 2020-09-17 -not -path "*/.virtfs_metadata/*" | xargs wc -l
import os
import os.path
from os import path
import pandas as pd
from stat import S_ISREG, ST_CTIME, ST_MODE
import time
from datetime import datetime
import re

 
dfs = []
rootDir = 'output3'

def contains_csvfile(fileList):
    for file in fileList:
        if ".csv" in file:
            return True
    return False

def get_csvfile(fileList):
    for file in fileList:
        if ".csv" in file:
            return file
    return None

for dirName, subdirList, fileList in os.walk(rootDir, topdown=False):
    if ".virtfs_metadata" in dirName:
        continue
    
    if not contains_csvfile(fileList):
        print("no CSV file found in", dirName)
        continue
        
    else:        
        
        csv_fname = get_csvfile(fileList)
        
        ### deprecated / unused
        # print('%s\t%s' % (dirName, fname))
        stat_file = os.stat(os.path.join(dirName, csv_fname))
        cdate = stat_file[ST_CTIME]
        creationtime_file = time.ctime(os.path.getmtime(os.path.join(dirName, csv_fname)))
        creationdate_file = datetime.fromtimestamp(os.path.getmtime(os.path.join(dirName, csv_fname))).date()
        targeted_date = datetime.strptime("2020-09-16", "%Y-%m-%d")
        # if creationdate_file.day != targeted_date.day:
        #    continue
        
        def is_file_empty(file_path):
                """ Check if file is empty by confirming if its size is 0 bytes"""
                # Check if file exist and it is empty
                return os.path.exists(file_path) and os.stat(file_path).st_size == 0
        if is_file_empty(os.path.join(dirName, csv_fname)):
            # print("empty CSV file")
            continue
        df = pd.read_csv(os.path.join(dirName, csv_fname))
        df['sampler'] = csv_fname
        
        DEFAULT_BARBARIK_SAMPLER = 9 # KUS is the default sampler
        # processing metainformation (README)
        if path.isfile(os.path.join(dirName, 'README.md')):
            with open(os.path.join(dirName, 'README.md'), 'r') as readme:
                content_readme = readme.read()
                # print(content_readme)
                m = re.search('ref_sampler=(\d+)', content_readme)
                if m is not None:
                    r_sampler = int(m.group(1))
                    print("ref sampler 6 found in", dirName)
                else:
                    r_sampler = DEFAULT_BARBARIK_SAMPLER
                    # print("default sampler is used... KUS", default_sampler)
                    # continue

                df['ref_sampler'] = r_sampler
    
        dfs.append(df)
        
        
        
        
        
  
            
        
            

# for df in dfs:
    # df['sampler'] = solver
#    if len(df['file']) == 1 and "FM-3" in df['file'][0]:
#        print(solver)
#        print(df['file'])
#        print(df['Uniform'].value_counts(dropna=False))
#        print()


all_results = pd.concat(dfs)

all_results.to_csv('all-results-sampler.csv')
all_results[['file', 'time', 'Uniform', 'Timeout', 'sampler']].to_csv('all-results-sampler-digest.csv')

ref sampler 6 found in output3/7ad65c6779df4c56b92744fe6774b853
ref sampler 6 found in output3/005ba415f14947538fe9bee45889a652
ref sampler 6 found in output3/76fb0c2a140e4488a145af756480e75e
ref sampler 6 found in output3/0c772ded839a45bdb7ae3d7bdc47dab8
ref sampler 6 found in output3/2d34749a12c1415497f18a3ac9fe3818
ref sampler 6 found in output3/3288481b31a14fbc852782ed8ac89ac3
ref sampler 6 found in output3/62d999abca304dbc8413f39214abcd87
ref sampler 6 found in output3/d17a8835acf84d928777732e33b6d1d6
ref sampler 6 found in output3/e23ffabec25f479dbcf3675613e30158
ref sampler 6 found in output3/77b5f59538374ff5a2b2ee315bf36634
no CSV file found in output3


In [2]:
all_results.drop_duplicates(subset=['file', 'sampler', 'Uniform', 'Timeout', 'ref_sampler'], inplace=True)
# all_results.query("sampler == 'Uniform-DistanceBasedSampling.csv'")

i = 0
df_summary = pd.DataFrame(columns=['sampler', "nbench", "nTrueUniform", "nFalseUniform", "nNAUniform"])
for s in all_results["sampler"].unique():
    nbench = len(all_results.query("sampler == '" + s + "'"))
    ntrueu = len(all_results.query("Uniform == True and sampler == '" + s + "'"))
    nfalseu = len(all_results.query("Uniform == False and sampler == '" + s + "'"))
    nnau = len(all_results.query("Uniform != True and Uniform != False and sampler == '" + s + "'"))
    print(s, nbench, ntrueu)
    df_summary.loc[i] = (s, nbench, ntrueu, nfalseu, nnau)
    i = i + 1

df_summary.to_csv('summary-uniformity-samplers.csv')
df_summary
#all_results.query("sampler == 'Uniform-SPUR.csv' and Uniform == False")

Uniform-SPUR.csv 193 58
Uniform-CustomSampler.csv 96 77
Uniform-DistanceBasedSampling.csv 264 5
Uniform-SMARCH.csv 274 0
Uniform-UniGen.csv 20 0
Uniform-QuickSampler.csv 266 0
Uniform-STS.csv 270 0
Uniform-UNIGEN2.csv 26 4
Uniform-UniGen3.csv 39 25
Uniform-KUS.csv 163 46


,sampler,nbench,nTrueUniform,nFalseUniform,nNAUniform
0,Uniform-SPUR.csv,193,58,27,108
1,Uniform-CustomSampler.csv,96,77,9,10
2,Uniform-DistanceBasedSampling.csv,264,5,163,96
3,Uniform-SMARCH.csv,274,0,175,99
4,Uniform-UniGen.csv,20,0,1,19
5,Uniform-QuickSampler.csv,266,0,248,18
6,Uniform-STS.csv,270,0,128,142
7,Uniform-UNIGEN2.csv,26,4,3,19
8,Uniform-UniGen3.csv,39,25,6,8
9,Uniform-KUS.csv,163,46,16,101


In [3]:
all_results.query("sampler == 'Uniform-KUS.csv' and Uniform == False")['file'].values #[['file', 'time', 'Uniform', 'Timeout', 'sampler']]

array(['/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_1_b12_2.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_1_b11_1.cnf',
       '/home/samplingfm/Benchmarks/V3/s1238a_3_2.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_2_b12_2.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_0_b12_1.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case120.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_2_b12_1.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_squaring5.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case14.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case35.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_squaring26.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case34.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case116.cnf',
       '/home/samplingfm/Benchmarks/V15/s526_15_7.cnf',
       '/

In [4]:
all_results.query("sampler == 'Uniform-SPUR.csv' and Uniform == False")['file'].values #[['file', 'time', 'Uniform', 'Timeout', 'sampler']]

array(['/home/samplingfm/Benchmarks/Blasted_Real/blasted_case35.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_squaring26.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case34.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case116.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_1_b12_2.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_squaring28.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_1_b11_1.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case123.cnf',
       '/home/samplingfm/Benchmarks/V3/s1238a_3_2.cnf',
       '/home/samplingfm/Benchmarks/V7/s27_new_7_4.cnf',
       '/home/samplingfm/Benchmarks/V7/s27_7_4.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_case_0_b11_1.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Real/blasted_squaring6.cnf',
       '/home/samplingfm/Benchmarks/V15/s526_15_7.cnf',
       '/home/samplingfm/Benchmarks/Blasted_Rea

In [5]:
all_results.query('ref_sampler == ' + str(6))

,file,time,cmd_output,err_output,Uniform,Timeout,sampler,ref_sampler
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,6.560,b'command: log /tmp/output.txt\ncommand: log /...,b'',False,False,Uniform-DistanceBasedSampling.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1.706,b'Experiment: 0 of 1\nconstantFactor:3...,b'',False,False,Uniform-QuickSampler.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,43000.041,NaN,NaN,NaN,True,Uniform-UniGen.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1306.412,b'Experiment: 0 of 1\nconstantFactor:3...,b'',False,False,Uniform-SMARCH.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1193.054,b'Experiment: 0 of 1\nconstantFactor:3...,b'',True,False,Uniform-CustomSampler.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,1850.024,b'Experiment: 0 of 1\nconstantFactor:3...,b'',True,False,Uniform-KUS.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,43000.005,NaN,NaN,NaN,True,Uniform-SPUR.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,16315.752,b'Experiment: 0 of 1\nconstantFactor:3...,b'',True,False,Uniform-UniGen3.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,0.807,b'Experiment: 0 of 1\nconstantFactor:3...,"b'Traceback (most recent call last):\n File ""...",NaN,False,Uniform-STS.csv,6.0
0,/home/samplingfm/Benchmarks/FeatureModels/FM-3...,38208.855,b'Experiment: 0 of 1\nconstantFactor:3...,b'',True,False,Uniform-UNIGEN2.csv,6.0
